# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!


  1. Use RAGAS to Generate Synthetic Data
  2. Load them into a LangSmith Dataset
  3. Evaluate our RAG chain against the synthetic test data
  4. Make changes to our pipeline
  5. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [16]:
#!pip install -qU ragas==0.2.10

In [17]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [18]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gkate\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gkate\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [19]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

LangChain API Key: ········


We'll also want to set a project name to make things easier for ourselves.

In [20]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [21]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [22]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [23]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [24]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [25]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [26]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node b65f84b3-a7ed-4b9e-a8fa-e97042b13346 does not have a summary. Skipping filtering.
Node 556318ae-3ff0-47f0-99a5-9f667c53be61 does not have a summary. Skipping filtering.
Node 1d348e1b-daeb-47b3-9a8a-f415ec11ea1e does not have a summary. Skipping filtering.
Node 7ca54312-593d-481e-b9f2-0d66b063b830 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 136)

We can save and load our knowledge graphs as follows.

In [27]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 136)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [28]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [29]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

* The three types of query synthesizers generate different kinds of questions for evaluating retrieval-augmented generation systems: the SingleHopSpecificQuerySynthesizer creates simple, direct questions answerable with a single fact; the MultiHopAbstractQuerySynthesizer generates more complex, conceptual questions that require connecting multiple pieces of information; and the MultiHopSpecificQuerySynthesizer produces detailed, focused questions that also require combining several facts, but in a concrete and specific way.
</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [31]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the EU's role in AI regulation?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The European Union's landmark AI Act of 2024 h...,single_hop_specifc_query_synthesizer
1,What are the main concerns about AI in the Phi...,[AI presents enormous opportunities for the Ph...,The main concerns about AI in the Philippines ...,single_hop_specifc_query_synthesizer
2,Does the Philippine Constitution say anything ...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,"The context mentions Article XIV, Section 10 o...",single_hop_specifc_query_synthesizer
3,What is the significance of ASI in AI policy d...,"[1 \na) Promote innovation, technological adva...",The context states that the Act shall regulate...,single_hop_specifc_query_synthesizer
4,How does artificial superintelligence (ASI) di...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial Superintelligence (ASI) refers to h...,single_hop_specifc_query_synthesizer
5,"How do guidelins and risk thresolds for AI, as...","[<1-hop>\n\n1 \nendorsements, voice recordings...","The guidelins and risk thresolds for AI, such ...",multi_hop_abstract_query_synthesizer
6,How does the authority to impose administrativ...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The authority to impose administrative penalti...,multi_hop_abstract_query_synthesizer
7,How does the composition and jurisdiction of t...,[<1-hop>\n\nAI presents enormous opportunities...,The NAIC has exclusive jurisdiction over all m...,multi_hop_abstract_query_synthesizer
8,How does the potential rise of ASI relate to A...,[<1-hop>\n\nAI presents enormous opportunities...,The context highlights that the global scienti...,multi_hop_specific_query_synthesizer
9,How does the Philippine Senate's proposed Arti...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The Philippine Senate's proposed Artificial In...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [32]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node b9f7bd3b-3cf1-4852-9d10-b56fd2b7bb8a does not have a summary. Skipping filtering.
Node 684b6fe2-73e7-46b2-ab10-22f80170f5ba does not have a summary. Skipping filtering.
Node e70f89d7-786c-4921-a39e-d326577c74ae does not have a summary. Skipping filtering.
Node 3778d627-b660-4d7a-bc4c-373a4c38df92 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [34]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What does the TWENTIETH CONGRESS do about AI p...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The TWENTIETH CONGRESS OF THE REPUBLIC OF THE ...,single_hop_specifc_query_synthesizer
1,ASI dangerous?,[AI presents enormous opportunities for the Ph...,The context mentions that the rise of Artifici...,single_hop_specifc_query_synthesizer
2,What was the significance of the TWENTIETH CON...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The TWENTIETH CONGRESS of the Republic of the ...,single_hop_specifc_query_synthesizer
3,How does the policy aim to protect Filipinos i...,"[1 \na) Promote innovation, technological adva...",The policy aims to promote the responsible and...,single_hop_specifc_query_synthesizer
4,How do the ai registration requirments and pen...,[<1-hop>\n\n1 \nSec. 12. Contents of AI Regist...,The ai registration and registry requirements ...,multi_hop_abstract_query_synthesizer
5,how do big language models and the rules about...,[<1-hop>\n\n1 \nground its responses in verifi...,big language models (LLMs) are a type of gener...,multi_hop_abstract_query_synthesizer
6,How do the AI registration requirements outlin...,[<1-hop>\n\n1 \nSec. 12. Contents of AI Regist...,The requirements in Sec. 12 mandate that each ...,multi_hop_abstract_query_synthesizer
7,How do the penalties for manipulatn and prohib...,"[<1-hop>\n\n1 \nendorsements, voice recordings...",The context shows that penalties for manipulat...,multi_hop_abstract_query_synthesizer
8,How do Sec. 11 and Sec. 16 relate to AI risk c...,[<1-hop>\n\n1 \nSec. 15. AI Ethics Review Boar...,Sec. 16 outlines the classification of AI syst...,multi_hop_specific_query_synthesizer
9,how dictt and dictt r related in ai polcy?,"[<1-hop>\n\n1 \nendorsements, voice recordings...",The context shows that the DICT (Department of...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [35]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [36]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [37]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [39]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [40]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [41]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [42]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [44]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [45]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is a fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment of three (3) years to ten (10) years, or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [46]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [47]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* qa_evaluator: This evaluator checks if the model’s answer is correct and relevant to the question, using an LLM to compare the generated answer with the reference answer.

* labeled_helpfulness_evaluator: This evaluator measures how helpful the model’s answer is to the user, specifically considering if the answer would assist the user when compared to the correct reference answer. It uses an LLM and a custom “helpfulness” criterion.
</div>

## LangSmith Evaluation

In [48]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'only-tail-28' at:
https://smith.langchain.com/o/7fc042b4-97e0-5f3f-b110-30a87bc46433/datasets/2e682626-7edb-4606-ad73-4627a7519185/compare?selectedSessions=6420f4ef-b693-4e88-b2e5-b754a874d2fb




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the AI registration process ensure co...,The AI registration process ensures compliance...,None,"The AI registration process, as outlined in th...",1,1,0,3.699511,8a60db1e-52a7-4dbe-a0d1-4ad4021b4b3d,52bdd5c1-ed73-402f-a5f0-69c572c4df04
1,How do the AI policies ensure ethical standard...,"Based on the provided context, AI policies ens...",None,The AI policies establish an AI Ethics Review ...,1,1,0,4.298804,01fea1e0-8654-41e6-bbcf-877f5789ff7d,be0840f8-4a08-469f-bbb5-f83c4775ccf7
2,how dictt and dictt r related in ai polcy?,I don't know.,None,The context shows that the DICT (Department of...,0,0,0,1.443747,ef57f6db-625c-4191-81a8-5953c8b09eb8,22dbacc7-374f-4a32-b3fa-9663d0e84c4c
3,How do Sec. 11 and Sec. 16 relate to AI risk c...,Based on the provided context:\n\n- Sec. 11 in...,None,Sec. 16 outlines the classification of AI syst...,1,0,0,4.376784,39f5df27-1cfc-4ade-8518-dc02dcbd99a5,2814a842-4b3f-44d8-9c67-d177251bd1b8
4,How do the penalties for manipulatn and prohib...,The penalties for manipulation and prohibited ...,None,The context shows that penalties for manipulat...,1,1,0,4.031389,b76c9779-ce7e-493b-b25f-97ce53cd2199,d042ead5-82ea-4801-8948-182d0322b1b2
5,How do the AI registration requirements outlin...,Based on the provided context:\n\n- The AI reg...,None,The requirements in Sec. 12 mandate that each ...,1,1,0,6.405471,de81a08c-8c8a-474b-a1b3-875683b7a9e2,04b37374-b1b9-4348-a6b7-0a53d50503f1
6,how do big language models and the rules about...,"Based on the provided context, the regulation ...",None,big language models (LLMs) are a type of gener...,1,0,0,7.055859,5c1a08cd-a740-4add-920c-4f4fcbb178e2,b623368f-969f-4017-ae13-9bbd7ae28c50
7,How do the ai registration requirments and pen...,"Based on the provided context, AI registration...",None,The ai registration and registry requirements ...,1,1,0,4.808705,4d5e512d-2ab4-46e9-9410-2ebb5cf8ba95,650783ab-04cf-486c-ae92-6fefbf09f120
8,How does the policy aim to protect Filipinos i...,The policy aims to protect Filipinos in the co...,None,The policy aims to promote the responsible and...,1,1,0,3.177548,89cca42e-bb2a-4170-be8c-e35b89feb354,bcc3ad6d-521f-433f-96c1-ac5e39dd115c
9,What was the significance of the TWENTIETH CON...,The TWENTIETH CONGRESS of the Republic of the ...,None,The TWENTIETH CONGRESS of the Republic of the ...,1,1,0,2.853124,ea05552d-c4c3-4ad0-a260-a471b42e5bb0,deae55bc-fc40-46ae-a781-70ce9872f29b


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [49]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [50]:
rag_documents = docs

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer: 

Modifying the chunk size changes how much text is grouped together and retrieved at once from the documents. If the chunks are too small, important context might be split up and missed by the model, leading to incomplete or less accurate answers. If the chunks are too large, irrelevant information may be included, making it harder for the model to find the most relevant details. Therefore, finding the right chunk size helps balance context and relevance, directly impacting the quality and performance of the application's responses.

</div>

In [52]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

Modifying the embedding model changes how the application represents and understands the meaning of text. Different embedding models vary in their ability to capture semantic relationships, context, and nuances in language. A more advanced or better-suited embedding model can improve how well the system matches questions to relevant information, leading to more accurate and helpful responses. While a less capable model might miss important connections, reducing performance. So, the choice of embedding model directly affects the quality of retrieval and, ultimately, the application's effectiveness.

</div>

In [53]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [54]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [55]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [56]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

"Thank you for your thoughtful question about the importance of the Philippines AI Bill. Based on the context provided, this bill is very important because it aims to carefully balance fostering technological innovation with ensuring that AI systems remain safe, ethical, transparent, and under meaningful human oversight. It recognizes that AI is still in its early stages and provides a general framework to encourage responsible and lawful AI development.\n\nMoreover, the bill envisions a future where AI supports Filipino creativity, helps address national development challenges, and protects the rights and welfare of every citizen. The State takes on the crucial responsibility to ensure AI is never used to commit crimes, violate rights, or cause harm, whether intentionally or by accident, while still promoting progress.\n\nIn essence, this bill is important because it seeks to guide the Philippines towards an inclusive, innovative, and secure digital future, safeguarding its people whi

Finally, we can evaluate the new chain on the same test set!

In [57]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'roasted-death-76' at:
https://smith.langchain.com/o/7fc042b4-97e0-5f3f-b110-30a87bc46433/datasets/2e682626-7edb-4606-ad73-4627a7519185/compare?selectedSessions=1ae46d35-af6d-4011-90ae-11d8a941b63f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the AI registration process ensure co...,Thank you for your thoughtful question. Based ...,None,"The AI registration process, as outlined in th...",1,1,1,8.406520,8a60db1e-52a7-4dbe-a0d1-4ad4021b4b3d,3f706e84-ad1a-4429-b0c9-fda616c14913
1,How do the AI policies ensure ethical standard...,Thank you for asking such an important questio...,None,The AI policies establish an AI Ethics Review ...,1,1,1,4.284182,01fea1e0-8654-41e6-bbcf-877f5789ff7d,73c390d5-134c-4455-8755-1668de1c10bb
2,how dictt and dictt r related in ai polcy?,Thank you for your question. It seems like you...,None,The context shows that the DICT (Department of...,0,0,1,3.419880,ef57f6db-625c-4191-81a8-5953c8b09eb8,2338466f-4849-47a5-b486-d6b9c6b8e75a
3,How do Sec. 11 and Sec. 16 relate to AI risk c...,Thank you for your thoughtful question. Based ...,None,Sec. 16 outlines the classification of AI syst...,0,0,1,7.057860,39f5df27-1cfc-4ade-8518-dc02dcbd99a5,c87d56cf-88ea-45e6-bae1-9796827b388c
4,How do the penalties for manipulatn and prohib...,Thank you for your thoughtful question. Based ...,None,The context shows that penalties for manipulat...,1,1,1,6.053378,b76c9779-ce7e-493b-b25f-97ce53cd2199,393a288d-fb15-4d32-a840-c59564204400
5,How do the AI registration requirements outlin...,Thank you for your thoughtful question. Based ...,None,The requirements in Sec. 12 mandate that each ...,1,1,1,6.373147,de81a08c-8c8a-474b-a1b3-875683b7a9e2,9da5d3e2-b289-4fe6-b6ad-a1ff803da69e
6,how do big language models and the rules about...,Thank you for your thoughtful question. It’s c...,None,big language models (LLMs) are a type of gener...,1,0,1,7.569376,5c1a08cd-a740-4add-920c-4f4fcbb178e2,c8fb6e3c-acf2-4f74-97f0-6103d49df7be
7,How do the ai registration requirments and pen...,Thank you for your thoughtful question. Based ...,None,The ai registration and registry requirements ...,1,1,1,18.560682,4d5e512d-2ab4-46e9-9410-2ebb5cf8ba95,b0814802-300f-46dc-83bb-3d5fde1d375c
8,How does the policy aim to protect Filipinos i...,Thank you for your thoughtful question. It’s c...,None,The policy aims to promote the responsible and...,1,1,1,4.237498,89cca42e-bb2a-4170-be8c-e35b89feb354,7546c2b1-461d-4cdb-8db0-3db91b3d10ee
9,What was the significance of the TWENTIETH CON...,Thank you for your thoughtful question. Based ...,None,The TWENTIETH CONGRESS of the Republic of the ...,1,1,1,5.061258,ea05552d-c4c3-4ad0-a260-a471b42e5bb0,e7e65527-ab00-44e7-8e64-e154553dfd7d


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

Changes in metrics occur because modifications to chunk size, embedding models, or prompt design directly impact how effectively the system retrieves and answers questions. For example, a better embedding model or optimal chunk size can improve faithfulness, answer relevancy, and correctness by enabling the system to find and use more relevant information. While, suboptimal settings may lower precision or recall, affecting these metrics negatively. Comparing screenshots of evaluation results before and after such changes visually highlights these metric differences, illustrating the impact of the adjustments on system performance.

</div>